# Generate data for all figures and export as CSV files for plotting in TikZ

### Subfunction to generate the grids to evaluate the models over

In [55]:
function create_grids(ca, V)
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur = Array(Any, (1*length(V),))
    x_grid_cur_dense = Array(Any, (1*length(Vdense),))
    iterind = 0;
    for j1 in product([ca], V)
        iterind += 1;
        x_grid_cur[iterind] = collect(j1)
    end
    iterind = 0;
    for j1 in product([ca], Vdense)
        iterind += 1;
        x_grid_cur_dense[iterind] = collect(j1)
    end
        
    return x_grid_cur, x_grid_cur_dense
end

create_grids (generic function with 1 method)

# Figure 1

In [56]:
# Figure 1 A - Drawn in TikZ

In [57]:
# Figure 1 B and C

# Load the BK functions, the x_grid(Ca, V) that we evaluate over and the initial condition phi0
include("../BK_functions/bk_setup_script.jl"); 

Vdense = minimum(V):3e-3:maximum(V) # voltage sampled every 3 millivolts for smooth lines

for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for original model
    tmp = BK_simulator(phi0, hcat(x_grid_cur...), model_id=10)
    writecsv("CSV/figure_1b_panel1_ca$(ca_str)um.csv", hcat(V[:]*1000, tmp[:]))
    
    # Save also the log10 values
    writecsv("CSV/figure_1c_panel1_ca$(ca_str)um.csv", hcat(V[:]*1000, log10(tmp[:])))
    
    # Densely sampled
    tmp = BK_simulator(phi0, hcat(x_grid_cur_dense...), model_id=10)
    writecsv("CSV/figure_1b_panel2_ca$(ca_str)um.csv", hcat(Vdense[:]*1000, tmp[:]))
    writecsv("CSV/figure_1c_panel2_ca$(ca_str)um.csv", hcat(Vdense[:]*1000, log10(tmp[:])))
end

# Figure 2

In [58]:
# Figure 2 A
tmp = load("saves/figure_A1_hess_"*"8"*".jld") # Example noisy fit with diverging parameters
phi1 = tmp["phi1_orig"] # Get the fitted parameter vector

# Output the simulated data point markers in a CSV format with: 
# first column - voltage (mV), second column - Popen, third column error top, fourth column

# Also output the example fit with just first col - voltage (mV), second col - Popen
# but with voltage sampled more densely to make it a continous line

for ca in Ca  
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for original model
    tmp = BK_simulator(phi0, hcat(x_grid_cur...), model_id=0)
    errbar = tmp.*0.1 # Generate the error bar for every data point.
    writecsv("CSV/figure_2a_panel1_ca$(ca_str)um.csv", hcat(V[:].*1000, tmp[:], errbar[:], errbar[:]))
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(phi1, hcat(x_grid_cur_dense...), model_id=0)
    writecsv("CSV/figure_2a_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

In [59]:
# Figure 2 B
# Output the original and the fitted parameter values (first and second column of CSV respectively)
writecsv("CSV/figure_2b_panel1.csv", [phi0 phi1])

# Figure 3

In [60]:
# Figure 3 A
# Table with theoretical lower bound on parameter uncertainties

# REF NEEDED
# CALCULATION NEEDED
# DERIVATION NEEDED

In [61]:
# Figure 3 B - Drawn in TikZ

In [62]:
# Figure 3 C
using JLD
outp = load("saves/figure3_smallVnorm.jld", "outp");
outp_log10 = load("saves/figure4.jld", "outp_log10");

writecsv("CSV/figure_3c_panel1_orig.csv", hcat(1:size(outp[3][1,1],1), log(outp[3][1,1])))
writecsv("CSV/figure_3c_panel1_log10.csv", hcat(1:size(outp_log10[3][1,1],1), log(outp_log10[3][1,1])))

# Figure 4

In [63]:
# Figure 4 A
# Cost for original space model
# writecsv("CSV/figure_4a_panel1.csv", hcat(0:size(outp[1],1), vcat(outp[2][1,1], outp[2][:,2])))
writecsv("CSV/figure_4a_panel1.csv", hcat(0:5, vcat(outp[2][1,1], outp[2][1:5,2])))

In [64]:
# Figure 4 B
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 B
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp[1][4,1]), hcat(x_grid_cur_dense...), model_id=13)
    writecsv("CSV/figure_4b_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

In [65]:
# Figure 4 C
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 B
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp[1][5,1]), hcat(x_grid_cur_dense...), model_id=14)
    writecsv("CSV/figure_4c_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

# Figure 5 / Figure A1

Show the MBAM reduction steps (in Figure 5 schematically, in Figure A1 with more details)

* Panel n - nth reduction containing each parameter curve

In [74]:
# Naming the original and reduced parameter vectors for each model reduction step

param_names = Array(Array{AbstractString},size(outp[1],1));
param_names[1] = ["L0", "zL", "J0", "zJ", "Kd", "C", "D", "E"]
param_names[2] = ["L0", "J0", "zJ", "Kd", "C", "D", "E"]
param_names[3] = ["J0", "zJ", "Kd", "LC","LD", "EL"];
param_names[4] = ["J0", "zJ", "Kd", "CE","LD"];
param_names[5] = ["LDJ", "zJ", "Kd", "CE"];
param_names[6] = ["LDJ", "zJ", "CEKd"];

In [76]:
for model_red = 1:size(outp[1],1)
    # Get the log param values during the reduction
    phi_red_time = outp[5][model_red]
    phi_red_vals = hcat(outp[6][model_red]...)
    phi_red_vals = phi_red_vals[1:div(size(phi_red_vals,1),2),:]
    
    # Find the converged time (the one after no parameters change)
    cutoff = 0
    for i1 = 2:size(phi_red_vals,2)
        if all(phi_red_vals[:,i1].==phi_red_vals[:,i1-1])
            cutoff = i1
            break
        end
    end
    
    # Produce the geodesic time - log(value) for each parameter for each reduction step
    for param_num in 1:size(phi_red_vals,1)
        writecsv("CSV/figure_5"*string("abcdef"[model_red])*"_panel1_param_$(param_names[model_red][param_num]).csv", 
            hcat(phi_red_time[1:cutoff], phi_red_vals[param_num,1:cutoff][:]))
    end
end

# Figure 6

In [69]:
# Figure 6 A
# Cost for log space model
writecsv("CSV/figure_6a_panel1.csv", hcat(0:4, vcat(outp_log10[2][1,1], outp_log10[2][1:4,2])))

In [70]:
# Figure 6 B
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 C
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp_log10[1][2,1]), hcat(x_grid_cur_dense...), model_id=52)
    writecsv("CSV/figure_6b_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

In [71]:
# Figure 6 C
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 C
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp_log10[1][3,1]), hcat(x_grid_cur_dense...), model_id=53)
    writecsv("CSV/figure_6c_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

# Figure 7

In [ ]:
# RELATIVE ERRORS TO BE RECALCULATED AND DERIVED